# **Burn Severity Map of Tenerife wildfire, Aug 2023**

<hr>

## About

### Notebook outline
- [1 - Load and browse the Sentinel-2 MSI data](#s2_load_21)
- [2 - Create natural colour composites](#s2_natural_21)
- [3 - Create false colour composites](#s2_false_21)
- [4 - Calculate the Normalized Burn Ratio (NBR) and Differenced NBR images](#s2_nbr_21)
- [5 - Create the burn severity map](#s2_burnseverity_21)

In [1]:
#!pip install numpy
#!pip install pandas
#!pip install matplotlib
#!pip install rasterio
#!pip install scikit-image
#!pip install sentinelsat
#!pip install geopandas

#### Load required libraries

In [7]:
#Python libraries

import os
import numpy as np
import pandas as pd
import glob as glob

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.cm import get_cmap
from matplotlib.axes import Axes
from skimage import exposure
import rasterio
from rasterio import plot
from rasterio.plot import show

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

import warnings
#warnings.simplefilter(action = "ignore", category = RuntimeWarning)
#warnings.simplefilter(action = "ignore", category = UserWarning)

<hr>

## <a id='s2_load_21'></a>1. Download the Sentinel-2 MSI data

In [3]:
api = SentinelAPI('USER', 'PASS', 'https://scihub.copernicus.eu/dhus',show_progressbars=True)

In [8]:
# Search by polygon, time, and SciHub query keywords
footprint = geojson_to_wkt(read_geojson('./TenerifeWildfire.geojson'))
products = api.query(footprint,
                     date=("20230814","20230825"),
                     platformname='Sentinel-2',
                     producttype= 'S2MSI2A')
                     #cloudcoverpercentage=(0, 50))

In [9]:
# download all results from the search
api.download_all(products)

# convert to Pandas DataFrame
products_df = api.to_dataframe(products)

# GeoJSON FeatureCollection containing footprints and metadata of the scenes
api.to_geojson(products)

# GeoPandas GeoDataFrame with the metadata of the scenes and the footprints as geometries
api.to_geodataframe(products)

# Get basic information about the product: its title, file size, MD5 sum, date, footprint and
# its download url
#api.get_product_odata(<product_id>)

# Get the product's full metadata available on the server
#api.get_product_odata(<product_id>, full=True)

,title,link,link_alternative,link_icon,summary,ondemand,generationdate,beginposition,endposition,ingestiondate,...,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,datastripidentifier,granuleidentifier,identifier,uuid,geometry
6e0d4a7e-1d83-4faf-9756-694a0378a6b7,S2B_MSIL2A_20230823T115219_N0509_R123_T28RCS_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-08-23T11:52:19.024Z, Instrument: MS...",false,2023-08-23 14:04:08,2023-08-23 11:52:19.024,2023-08-23 11:52:19.024,2023-08-23 17:14:25.170,...,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_2BPS_20230823T140408_S2023...,S2B_OPER_MSI_L2A_TL_2BPS_20230823T140408_A0337...,S2B_MSIL2A_20230823T115219_N0509_R123_T28RCS_2...,6e0d4a7e-1d83-4faf-9756-694a0378a6b7,"MULTIPOLYGON (((-17.03241 27.92215, -15.91681 ..."
61524154-a663-4ce6-b948-1621f5c869cb,S2A_MSIL2A_20230818T115221_N0509_R123_T28RCS_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-08-18T11:52:21.024Z, Instrument: MS...",false,2023-08-18 16:27:59,2023-08-18 11:52:21.024,2023-08-18 11:52:21.024,2023-08-18 17:58:23.834,...,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_2APS_20230818T162759_S2023...,S2A_OPER_MSI_L2A_TL_2APS_20230818T162759_A0425...,S2A_MSIL2A_20230818T115221_N0509_R123_T28RCS_2...,61524154-a663-4ce6-b948-1621f5c869cb,"MULTIPOLYGON (((-17.03241 27.92215, -15.91681 ..."


In [10]:
products_df = api.to_dataframe(products)
products_df.head()

,title,link,link_alternative,link_icon,summary,ondemand,generationdate,beginposition,endposition,ingestiondate,...,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,datastripidentifier,granuleidentifier,identifier,uuid
6e0d4a7e-1d83-4faf-9756-694a0378a6b7,S2B_MSIL2A_20230823T115219_N0509_R123_T28RCS_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-08-23T11:52:19.024Z, Instrument: MS...",false,2023-08-23 14:04:08,2023-08-23 11:52:19.024,2023-08-23 11:52:19.024,2023-08-23 17:14:25.170,...,GS2B_20230823T115219_033754_N05.09,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_2BPS_20230823T140408_S2023...,S2B_OPER_MSI_L2A_TL_2BPS_20230823T140408_A0337...,S2B_MSIL2A_20230823T115219_N0509_R123_T28RCS_2...,6e0d4a7e-1d83-4faf-9756-694a0378a6b7
61524154-a663-4ce6-b948-1621f5c869cb,S2A_MSIL2A_20230818T115221_N0509_R123_T28RCS_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-08-18T11:52:21.024Z, Instrument: MS...",false,2023-08-18 16:27:59,2023-08-18 11:52:21.024,2023-08-18 11:52:21.024,2023-08-18 17:58:23.834,...,GS2A_20230818T115221_042591_N05.09,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_2APS_20230818T162759_S2023...,S2A_OPER_MSI_L2A_TL_2APS_20230818T162759_A0425...,S2A_MSIL2A_20230818T115221_N0509_R123_T28RCS_2...,61524154-a663-4ce6-b948-1621f5c869cb
